# 摘要

本專案目標在於，將過往會使用到的圖形資料，移除掉所有數值以後，僅保留各個欄位的變數名稱，基本資料(學校代碼、學校名稱 等)。

合併成一個 excel，並以分頁區隔不同的圖形所用之資料。

進行此行為的原因在於，當能夠將所有資料放在一起，就可以單從一個檔案作資料指定，進而可省去一些後續讀取資料所需的時間，降低時間機會成本。

# 採用 os 遍歷所有檔案，並合併。
遍歷資料夾 : 111 年大專校院公開平台數據計算_完成檔

In [ ]:
import os
import re

In [ ]:
data_nas_folder_path = os.path.join(os.path.dirname(os.getcwd()), "大專校院公開平台數據計算_參考版")

data_nas_path_generator = os.walk(data_nas_folder_path)

data_nas_path_list = []
for dirpath, dirname, files in data_nas_path_generator :
    for file in files : 
        data_path = os.path.join(dirpath, file)
        data_nas_path_list.append(data_path)
print(data_nas_path_list)


In [ ]:
import pandas
import numpy
import traceback

In [ ]:
# 建立一個參考用的 DataFrame ，之後會使用到這個，第一次看可以先略過。
base_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "大專校院公開平台數據計算_參考版"), "圖2-2-8日間學制學士班及專科延修人數比率（%）.xlsx")
base_data = pandas.read_excel(base_data_path, sheet_name= "最終結果")

# 創建 儲存錯誤資訊的 list。
error_list = []

# 載入填值表。
reference_excel_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "填值表.xlsx")
reference_excel = pandas.read_excel(reference_excel_path)

# 創建一個新的 Excel Writer 對象
writer_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step2_data_check.xlsx")
with pandas.ExcelWriter(writer_path, engine="openpyxl") as writer:
    # 從遍歷的 data_nas_list 中，採用 for-loop 合併。
    for data_path in data_nas_path_list : 
        try :
            data = pandas.read_excel(data_path, sheet_name = "最終結果") # K

            # 將欄位名稱統一，並使學校代碼轉換為字串。
            if "統計處學校代碼" in data.columns : 
                data = data.rename(columns= {"統計處學校代碼" : "學校代碼"})
            
            if "學校代碼" not in data.columns : 
                # 因為之後都是用學校代碼做數值的填入，是故，如果現在的檔案，不存在學校代碼，就無法填入資料。
                # 創建一個 學校代碼 欄位，並以 data_base 之數值填入。
                data.insert(0, "學校代碼", numpy.nan)

                # 從 目前 data 中，尋找與 base_data 具有一樣學校名稱的欄位，將 目前 data 中之學校代碼的位置，填上正確的學校代碼。
                for sch_name in list(dict.fromkeys(data["學校名稱"])) :
                    try : 
                        data.loc[data["學校名稱"] == sch_name, "學校代碼"] = base_data[base_data["學校名稱"] == sch_name]["統計處學校代碼"].values[0]

                    except Exception as e : 
                        # print("區塊一")
                        # print(str(e))
                        pass
                
                # 檢查 目前的 data 是否有學校的學校代碼為遺漏值
                if data["學校代碼"].isnull().sum() != 0 : 

                    file_name = os.path.splitext(os.path.basename(data_path))[0]
                    for var in data[data["學校代碼"].isna()]["學校名稱"].values : 
                        if var == "中信金融管理學院" : 
                            data.loc[data["學校名稱"] == var, "學校代碼"] = "1125" # 中信金融管理學院，在 base_data 中沒有辦法找到對應的學校代碼，此處直接寫入。
                        else : 
                            error_element = {"錯誤類型" : "遺漏值", "分頁名稱" : file_name, "學校名稱" : var}
                            error_list.append(error_element)
                    
            # 將 學校代碼的型別轉為字串。
            data["學校代碼"] = data["學校代碼"].astype(str)

            # 尋找 data 中，學校類別的欄位。這個欄位的下一個欄位，就是變數欄位。
            sch_classfication_index = data.columns.get_loc("學校類別")
            
            # 將每一個數值欄位 (如 速動比_101_學年度)，欄位名稱更改為 101，依此類推。採用正則表達式做數值選取。
            rename_list = []
            columns_list = data.iloc[:, sch_classfication_index + 1 : ].columns # 將數值欄位之欄位名稱都蒐集在一個 list。

            # 採用正則表達式，抓取 每個欄位的學年度部分。(如 速動比_101_學年度) 其中的 101。並存入 rename_list 之中。
            for column in columns_list : 
                sch_year_text = re.search(r'\d{3}', str(column)).group(0)
                rename_list.append(sch_year_text)

            # 將 columns_list、rename_list，組合成 dict。
            rename_dict = dict(zip(columns_list, rename_list))

            # 將欄位名稱全部改名。
            data = data.rename(columns= rename_dict)

            # 根據填值表，將變數名稱更改。
            column_name = reference_excel[reference_excel["使用圖型"] == os.path.splitext(os.path.basename(data_path))[0]]["變數名稱"].values[0]
            # 每一個分頁的第五行開始，皆是數值變數。
            data_column_list = data.columns[4:]

            # 欄位改名之結果 list。
            rename_list = [column_name + "_" + str(data_column) for data_column in data_column_list]

            rename_dict = dict(zip(data_column_list, rename_list))

            data = data.rename(columns = rename_dict)

            # 遺漏值以 NA 字串取代。
            data = data.fillna("NA")

            # 將每一個 Data 依序寫入各個分頁，分頁名子 data_path 的檔案名稱命名。
            file_name = os.path.splitext(os.path.basename(data_path))[0]
            data.to_excel(writer, sheet_name = file_name, index=False)


        except Exception as e: 
            print("格式不同之檔案")
            print(data_path)
            print(traceback.print_exc())
            print(type(columns_list[0]))



In [ ]:
error_data = pandas.DataFrame(error_list)
print(error_data)

In [ ]:
file_path = 'data/file.xlsx'
file_name, file_extension = os.path.splitext(os.path.basename(file_path))
print(file_name)

# 測試區